In [ ]:
from transitions import Machine
from transitions.extensions import GraphMachine
from main.models import SessionLocal, Pitch, Pitcher

In [ ]:
session = SessionLocal()

In [ ]:
test = (
    session.query(Pitch)
    .join(Pitcher, Pitch.pitcher_id == Pitcher.pitcher_id)
    .filter(Pitcher.pitcher_name == "Luis Severino")
    .order_by(Pitch.game_id.desc(), Pitch.pitch_number.asc()))

In [ ]:
for entry in test:
    print(f"{entry.game_id},{entry.pitch_type}, {entry.pitch_number}")